In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../")

In [ ]:
import cv2
import scipy
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares

from stereoVO.structures import StateBolts, VO_StateMachine
from stereoVO.datasets import KittiDataset
from stereoVO.configs import yaml_parser
from stereoVO.optimization import get_minimization
from stereoVO.geometry import (DetectionEngine, 
                               filter_matching_inliers, 
                               triangulate_points, 
                               filter_triangulated_points,
                               project_points)

In [ ]:
PATH = '../../KITTI/KITTI_gray/dataset/sequences/00/'
CONFIG_PATH =  '../configs/params.yaml'

**Initialise the dataset**

In [ ]:
dataset = KittiDataset(PATH)

dataset.intrinsic = dataset.camera_intrinsic()

**Load the Configs**

In [ ]:
params = yaml_parser(CONFIG_PATH)

**Initialise the State**

In [ ]:
state_num = 0

left_frame, right_frame, ground_truth = dataset[state_num]

#initialise the state
prevState = VO_StateMachine(state_num)

#set frame state
prevState.frames = left_frame, right_frame

**Detction Engine, Matching and Triangulation for first frame**

In [ ]:
detection_engine = DetectionEngine(prevState.frames.left, prevState.frames.right, params)

prevState.matchedPoints, prevState.keypoints, prevState.descriptors = detection_engine.get_matching_keypoints()

prevState.inliers, mask_epipolar = filter_matching_inliers(prevState.matchedPoints.left,
                                            prevState.matchedPoints.right,
                                            dataset.intrinsic,
                                            params)

prevState.pts3D, reproj_error = triangulate_points(prevState.inliers.left,
                                    prevState.inliers.right,
                                    dataset.PL,
                                    dataset.PR)

args_triangulation = params.geometry.triangulation
prevState.pts3D_Filter, maskTriangulationFilter, ratioFilter = filter_triangulated_points(prevState.pts3D, reproj_error, **args_triangulation)

prevState.InliersFilter = prevState.inliers.left[maskTriangulationFilter], prevState.inliers.right[maskTriangulationFilter]
prevState.ratioTriangulationFilter = ratioFilter

**Initialise the Second State and set it as currState**

In [ ]:
state_num = 1

left_frame, right_frame, ground_truth = dataset[state_num]

#initialise the state
currState = VO_StateMachine(state_num)

#set frame state
currState.frames = left_frame, right_frame

In [ ]:
detection_engine = DetectionEngine(currState.frames.left, currState.frames.right, params)

currState.matchedPoints, currState.keypoints, currState.descriptors = detection_engine.get_matching_keypoints()

currState.inliers, mask_epipolar = filter_matching_inliers(currState.matchedPoints.left,
                                            currState.matchedPoints.right,
                                            dataset.intrinsic,
                                            params)

currState.pts3D, reproj_error = triangulate_points(currState.inliers.left,
                                    currState.inliers.right,
                                    dataset.PL,
                                    dataset.PR)

args_triangulation = params.geometry.triangulation
currState.pts3D_Filter, maskTriangulationFilter, ratioFilter = filter_triangulated_points(currState.pts3D, reproj_error, **args_triangulation)

currState.InliersFilter = currState.inliers.left[maskTriangulationFilter], currState.inliers.right[maskTriangulationFilter]
currState.ratioTriangulationFilter = ratioFilter

In [ ]:
# tracker = TrackingEngine(prev_frames, curr_frames, prevState.InliersFilter)

In [ ]:
prevFrames = prevState.frames
currFrames = currState.frames
prevInliers = prevState.InliersFilter

**Feature Tracking from prev state to current state**

In [ ]:
lk_params = dict(winSize  = (21, 21), 
                 maxLevel = 5,
                 criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.03))

dist_x_y = lambda x,y:np.sqrt(np.sum((x - y)**2, axis=1))

def feature_tracking(imageRef, imageCur, pointsRef):
    
    """
    To Do : Add docstring for the function here.
    """
    
    assert len(pointsRef.shape)==2 and pointsRef.shape[1]==2
    
    pointsRef = pointsRef.reshape(-1,1,2).astype('float32')
    points_t0_t1, mask_t0_t1, err = cv2.calcOpticalFlowPyrLK(imageRef, imageCur, pointsRef, None, **lk_params)
    
    pointsRef = pointsRef.reshape(-1,2)
    points_t0_t1 = points_t0_t1.reshape(-1,2)
    mask_t0_t1 = mask_t0_t1.flatten().astype(bool)
       
#     pointsRef = pointsRef[mask_t0_t1]
#     points_t0_t1 = points_t0_t1[mask_t0_t1]
    
    return pointsRef, points_t0_t1, mask_t0_t1

In [ ]:
pointsFilterLeft, pointsTrackedLeft, maskTrackingLeft = feature_tracking(prevFrames.left,
                                                                        currFrames.left,
                                                                        prevInliers.left)

In [ ]:
pointsFilterRight, pointsTrackedRight, maskTrackingRight = feature_tracking(prevFrames.right,
                                                                           currFrames.right,
                                                                           prevInliers.right)

In [ ]:
# Joint index and select only good tracked points
maskTracking = np.logical_and(maskTrackingLeft, maskTrackingRight)
pointsTracked = StateBolts(pointsTrackedLeft[maskTracking], pointsTrackedRight[maskTracking])

In [ ]:
# Remove non-valid points from inliers filtered in prev state
prevStateInliers = StateBolts(prevInliers.left[maskTracking], prevInliers.right[maskTracking])
pts3D_TrackingFilter = prevState.pts3D_Filter[maskTracking]

In [ ]:
(_, __ ) , mask_tracking_epipolar_left = filter_matching_inliers(prevStateInliers.left, pointsTracked.left, dataset.intrinsic, params)

In [ ]:
(_, __ ) , mask_tracking_epipolar_right = filter_matching_inliers(prevStateInliers.right, pointsTracked.right, dataset.intrinsic, params)

In [ ]:
mask_tracking_epipolar = np.logical_and(mask_tracking_epipolar_left, mask_tracking_epipolar_right)

In [ ]:
currState.pointsTracked = (pointsTracked.left[mask_tracking_epipolar], pointsTracked.right[mask_tracking_epipolar])
prevState.inliersTrackingFilter =  (prevStateInliers.left[mask_tracking_epipolar], prevStateInliers.right[mask_tracking_epipolar])
prevState.pts3D_TrackingFilter = pts3D_TrackingFilter[mask_tracking_epipolar]                         

**P3P Solver**

In [ ]:
args_pnpSolver = params.geometry.pnpSolver


for i in range(args_pnpSolver.numTrials):
    
    retval, r_vec1, t_vec1, idxPose = cv2.solvePnPRansac(prevState.pts3D_TrackingFilter,
                                                       currState.pointsTracked.left,
                                                       dataset.intrinsic,
                                                       None,
                                                       iterationsCount=args_pnpSolver.numTrials,
                                                       reprojectionError=args_pnpSolver.reprojectionError,
                                                       confidence=args_pnpSolver.confidence,
                                                       flags=cv2.SOLVEPNP_P3P)
    
    
    r_mat, _ = cv2.Rodrigues(r_vec1)
    
    # r_vec and t_vec obtained are in camera coordinate frames
    # we need to convert these matrix in world coordinates system
    # or we need to transforms the matrix from currState to prevState
    r_mat = r_mat.T
    t_vec = -r_mat.T @ t_vec1
    
    idxPose = idxPose.flatten().astype(bool)
    
    ratio = sum(idxPose)/len(idxPose)
    scale = np.linalg.norm(t_vec)
    
    if scale<args_pnpSolver.deltaT and ratio>args_pnpSolver.minRatio:
        print("Scale of translation of camera     : {}".format(scale))
        print("Solution obtained in P3P Iteration : {}".format(i+1))
        print("Ratio of Inliers                   : {}".format(ratio))
        break
    else:
        print("Warning : Max Iter : {} reached, still large position delta produced".format(i))

Scale of translation of camera     : 0.6764176064655109
Solution obtained in P3P Iteration : 1
Ratio of Inliers                   : 1.0


**Non-linear Least Square Optimization of Pose**

In [ ]:
optimisation=True

if optimisation:
    
    # Convert the matrix from world coordinates(prevState) to camera coordinates (currState)
    r_mat = r_mat.T
    t_vec = -r_mat.T@t_vec
    r_vec, _ = cv2.Rodrigues(r_mat)
    
    # Prepare an initial set of parameters to the optimizer
    doF = np.concatenate((r_vec, t_vec)).flatten()
 
    # Prepare the solver for minimization
    optRes = least_squares(get_minimization, doF, method='lm', max_nfev=2000,
                            args=(prevState.pts3D_TrackingFilter, 
                                  currState.pointsTracked.left, 
                                  currState.pointsTracked.right, 
                                  dataset.PL,dataset.PR))
    
    # r_vec and t_vec obtained are in camera coordinate frames (currState)
    # we need to convert these matrix to world coordinates system (prevState)
    opt_rvec_cam = (optRes.x[:3]).reshape(-1,1)
    opt_tvec_cam = (optRes.x[3:]).reshape(-1,1)
    opt_rmat_cam,_ = cv2.Rodrigues(opt_rvec_cam)
    
    # Obtain the pose of the camera (wrt state of the previous camera)
    r_mat = opt_rmat_cam.T
    t_vec = -opt_rmat_cam.T @ opt_tvec_cam

NameError: name 'project_points' is not defined

In [ ]:
# Upating the pose of the camera of currState 
# According to the Vo tutorial by Dacvid Scaramuzza 
# C_n = C_n-1 * dTn-1
# where the dTn-1 is in the coordinate frame of the second camera
# Link : http://rpg.ifi.uzh.ch/docs/VO_Part_I_Scaramuzza.pdf